In [1]:
import pandas as pd
import numpy as np

df_wind_solar = pd.read_csv("Datasets\Dataframes_one_ex_var\ENTSOE BE Wind and Solar Generation 2023 hourly.csv")
df_wind_solar = df_wind_solar[df_wind_solar.columns[1:5]]
print(df_wind_solar.shape)
df_wind_solar.index = pd.date_range("2023-01-01 00:00:00", "2023-12-31 23:00:00", freq="H")
df_wind_solar["Total wind"] = df_wind_solar.fillna(0)["Wind Onshore  - Actual Aggregated [MW]"] + df_wind_solar.fillna(0)["Wind Offshore  - Actual Aggregated [MW]"]
df_wind_solar = df_wind_solar.drop(columns=["MTU", "Wind Onshore  - Actual Aggregated [MW]", "Wind Offshore  - Actual Aggregated [MW]"]) 
df_wind_solar.head()

(8760, 4)


,Solar - Actual Aggregated [MW],Total wind
2023-01-01 00:00:00,0.0,4193.0
2023-01-01 01:00:00,0.0,4175.0
2023-01-01 02:00:00,0.0,4167.0
2023-01-01 03:00:00,0.0,4087.0
2023-01-01 04:00:00,0.0,4105.0


In [2]:
df_load = pd.read_csv("Datasets\Dataframes_one_ex_var\ENTSOE BE Load Actual 2023.csv")
# df_load["Time (CET/CEST)"].value_counts() == 2
df_load["Time (CET/CEST)"] = pd.date_range("2023-01-01 00:00:00", "2023-12-31 23:45:00", freq="15min")
df_load["day_of_year"] = df_load["Time (CET/CEST)"].dt.dayofyear
df_load["hour"] = df_load["Time (CET/CEST)"].dt.hour
df_load.head()
df_load = df_load[df_load.columns[2:]].groupby(["day_of_year", "hour"], as_index=False).mean()
df_load.index = pd.date_range("2023-01-01 00:00:00", "2023-12-31 23:00:00", freq="H")
# df_load.index = pd.date_range("2023-01-01 00:00:00", "2023-12-31 23:00:00", freq="H")
df_load = df_load.drop(columns=["day_of_year", "hour"])
df_load

,Actual Total Load [MW] - BZN|BE
2023-01-01 00:00:00,7378.50
2023-01-01 01:00:00,7083.00
2023-01-01 02:00:00,6820.25
2023-01-01 03:00:00,6588.50
2023-01-01 04:00:00,6430.75
...,...
2023-12-31 19:00:00,8636.00
2023-12-31 20:00:00,8244.50
2023-12-31 21:00:00,8012.75
2023-12-31 22:00:00,7869.50


In [3]:
import pandas as pd
def clean_dataframe(PATH, year):
    df_generation = pd.read_csv(PATH)
    df_generation.columns[2:]
    df_generation['Total generation'] = df_generation[df_generation.columns[2:]].sum(axis=1)
    df_generation = df_generation.drop(columns=df_generation.columns[2:-1])
    df_generation.index = pd.date_range(f"{year}-01-01 00:00:00", f"{year}-12-31 23:00:00", freq="H")
    df_generation = df_generation.drop(columns=df_generation.columns[0:2])
    return df_generation
df_2018 = clean_dataframe("Datasets\Dataframes_one_ex_var\All generation 2018.csv", year="2018")
df_2019 = clean_dataframe("Datasets\Dataframes_one_ex_var\All generation 2019.csv", year="2019")
df_2020 = clean_dataframe("Datasets\Dataframes_one_ex_var\All generation 2020.csv", year="2020")
df_2021 = clean_dataframe("Datasets\Dataframes_one_ex_var\All generation 2021.csv", year="2021")
df_2022 = clean_dataframe("Datasets\Dataframes_one_ex_var\All generation 2022.csv", year="2022")
df_2023 = clean_dataframe("Datasets\Dataframes_one_ex_var\ENTSOE BE Generation actual all types 2023.csv", year="2023")

df_generation = pd.concat([df_2018, df_2019, df_2020, df_2021, df_2022, df_2023])
df_2018.shape, df_2019.shape, df_2020.shape, df_2021.shape, df_2022.shape, df_2023.shape
df_generation.shape


C:\Users\r0776327\AppData\Local\Temp\ipykernel_1988\3637183246.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_generation['Total generation'] = df_generation[df_generation.columns[2:]].sum(axis=1)
C:\Users\r0776327\AppData\Local\Temp\ipykernel_1988\3637183246.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  df_generation['Total generation'] = df_generation[df_generation.columns[2:]].sum(axis=1)
C:\Users\r0776327\AppData\Local\Temp\ipykernel_1988\3637183246.py:5: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns

(52584, 1)

In [4]:
df_generation.shape, df_load.shape, df_wind_solar.shape

((52584, 1), (8760, 1), (8760, 2))

In [5]:
df_generation.to_csv("Datasets/ENTSOE_BE_Generation_actual_sum.csv")
df_load.to_csv("Datasets/ENTSOE_BE_Load_Actual_2023.csv")
df_wind_solar.to_csv("Datasets/ENTSOE_BE_Wind_and_Solar_Generation_2023.csv")

In [6]:
df_load_2018_2022 = pd.read_csv("Datasets\Dataframes_one_ex_var\ModelBELoad_dataframe.csv", index_col=0)["BE Load[MW]"]
df_load_2018_2022 = df_load_2018_2022.to_frame('BE Load')
df_load.columns = ["BE Load"]
df_load_total = pd.concat([df_load_2018_2022, df_load])
df_load_total.index = pd.to_datetime(df_load_total.index)
df_load_total

,BE Load
2018-01-01 00:00:00,8781.25
2018-01-01 01:00:00,8531.50
2018-01-01 02:00:00,8336.50
2018-01-01 03:00:00,8246.50
2018-01-01 04:00:00,8223.25
...,...
2023-12-31 19:00:00,8636.00
2023-12-31 20:00:00,8244.50
2023-12-31 21:00:00,8012.75
2023-12-31 22:00:00,7869.50


In [7]:
df_generation.columns = ["BE Generation"]
df_generation.shape

(52584, 1)

In [8]:
df_generation_load = pd.merge(df_generation, df_load_total, left_index=True, right_index=True, how="inner")
df_generation_load

,BE Generation,BE Load
2018-01-01 00:00:00,9572.0,8781.25
2018-01-01 01:00:00,9484.0,8531.50
2018-01-01 02:00:00,9584.0,8336.50
2018-01-01 03:00:00,9377.0,8246.50
2018-01-01 04:00:00,9847.0,8223.25
...,...,...
2023-12-31 19:00:00,10710.0,8636.00
2023-12-31 20:00:00,10704.0,8244.50
2023-12-31 21:00:00,10332.0,8012.75
2023-12-31 22:00:00,10086.0,7869.50


In [9]:
df_wind_solar.columns = ["BE Solar", "BE Wind"]
df_wind = pd.read_csv("Datasets\Dataframes_one_ex_var\ModelWind_dataframe.csv",index_col=0)["BE Wind"]
df_solar = pd.read_csv("Datasets\Dataframes_one_ex_var\ModelSolar_dataframe.csv",index_col=0)["BE Solar"]
df_wind.index = pd.to_datetime(df_wind.index)
df_solar.index = pd.to_datetime(df_solar.index)
merged_df = pd.merge(df_solar, df_wind, left_index=True, right_index=True)
df_wind_solar = pd.concat([merged_df, df_wind_solar])
df_wind_solar

,BE Solar,BE Wind
2018-01-01 00:00:00,0.0,2253.0
2018-01-01 01:00:00,0.0,2247.0
2018-01-01 02:00:00,0.0,2220.0
2018-01-01 03:00:00,0.0,2174.0
2018-01-01 04:00:00,0.0,2123.0
...,...,...
2023-12-31 19:00:00,0.0,4331.0
2023-12-31 20:00:00,0.0,4261.0
2023-12-31 21:00:00,0.0,4301.0
2023-12-31 22:00:00,0.0,4301.0


In [10]:
df_generation_load_wind_solar = pd.merge(df_generation_load, df_wind_solar, left_index=True, right_index=True, how="inner")
df_generation_load_wind_solar

,BE Generation,BE Load,BE Solar,BE Wind
2018-01-01 00:00:00,9572.0,8781.25,0.0,2253.0
2018-01-01 01:00:00,9484.0,8531.50,0.0,2247.0
2018-01-01 02:00:00,9584.0,8336.50,0.0,2220.0
2018-01-01 03:00:00,9377.0,8246.50,0.0,2174.0
2018-01-01 04:00:00,9847.0,8223.25,0.0,2123.0
...,...,...,...,...
2023-12-31 19:00:00,10710.0,8636.00,0.0,4331.0
2023-12-31 20:00:00,10704.0,8244.50,0.0,4261.0
2023-12-31 21:00:00,10332.0,8012.75,0.0,4301.0
2023-12-31 22:00:00,10086.0,7869.50,0.0,4301.0


In [11]:
# check if any columns in df_generation_load_wind_solar have missing values
df_generation_load_wind_solar.isnull().sum()

BE Generation    0
BE Load          1
BE Solar         1
BE Wind          0
dtype: int64

In [12]:
df_generation_load_wind_solar.fillna(0, inplace=True)

In [13]:
# return the rows with missing values
df_generation_load_wind_solar[df_generation_load_wind_solar.isnull().any(axis=1)]

,BE Generation,BE Load,BE Solar,BE Wind


In [14]:
df_price = pd.read_csv("Datasets/full_2018_2023_prices.csv", index_col=0)
df_price.index = pd.to_datetime(df_price.index)

df_generation_load_wind_solar_price = pd.merge(df_generation_load_wind_solar, df_price, left_index=True, right_index=True, how="inner")
df_generation_load_wind_solar_price

,BE Generation,BE Load,BE Solar,BE Wind,Price
2018-01-01 00:00:00,9572.0,8781.25,0.0,2253.0,4.74
2018-01-01 01:00:00,9484.0,8531.50,0.0,2247.0,3.66
2018-01-01 02:00:00,9584.0,8336.50,0.0,2220.0,1.26
2018-01-01 03:00:00,9377.0,8246.50,0.0,2174.0,-20.10
2018-01-01 04:00:00,9847.0,8223.25,0.0,2123.0,-31.82
...,...,...,...,...,...
2023-12-31 19:00:00,10710.0,8636.00,0.0,4331.0,20.26
2023-12-31 20:00:00,10704.0,8244.50,0.0,4261.0,19.65
2023-12-31 21:00:00,10332.0,8012.75,0.0,4301.0,9.00
2023-12-31 22:00:00,10086.0,7869.50,0.0,4301.0,10.68


In [ ]:
df_generation_load_wind_solar_price.to_csv("Datasets/Price_Generation_Load_Solar_Wind.csv")